In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import matplotlib.ticker as mtick
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
from shapely.geometry import Polygon

import shapely

In [2]:
df = pd.read_parquet("C:/Users/jorda/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [3]:
# First Dataset [Use Zip Code]
# No geographic values missing, no shapefiles missing, post 2000

In [4]:
df_geographic_unique = df[['state', 'reportedZipCode', 'countyCode', 'censusTract', 'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss']].drop_duplicates()

In [5]:
# Defining conditions
condition1 = (df_geographic_unique['latitude'].notna() & 
              df_geographic_unique['censusBlockGroupFips'].notna() & 
              df_geographic_unique['countyCode'].notna() & 
              df_geographic_unique['reportedZipCode'].isna())

condition2 = (df_geographic_unique['latitude'].notna() & 
              df_geographic_unique['censusBlockGroupFips'].notna() & 
              df_geographic_unique['countyCode'].notna() & 
              df_geographic_unique['reportedZipCode'].notna()&
              (df_geographic_unique['yearOfLoss'] < 2000))

# Filtering the dataframe based on the combined conditions
df_geographic_unique = df_geographic_unique[condition1 | condition2]

In [6]:
df_geographic_unique.head()

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss
0,CA,92056.0,6073.0,6.073019e+09,6.073019e+10,33.2,-117.3,1998
2,FL,32566.0,12113.0,1.211301e+10,1.211301e+11,30.4,-86.9,1998
3,SC,29902.0,45013.0,4.501300e+10,4.501300e+11,32.4,-80.7,1994
4,FL,32940.0,12009.0,1.200906e+10,1.200906e+11,28.3,-80.7,1996
5,VA,23451.0,51810.0,5.181004e+10,5.181004e+11,36.9,-76.0,1998


In [7]:
print(sum(df_geographic_unique['state'].isna()))
print(sum(df_geographic_unique['reportedZipCode'].isna()))
print(sum(df_geographic_unique['countyCode'].isna()))
print(sum(df_geographic_unique['censusTract'].isna()))
print(sum(df_geographic_unique['censusBlockGroupFips'].isna()))
print(sum(df_geographic_unique['latitude'].isna()))
print(sum(df_geographic_unique['yearOfLoss'].isna()))

0
9356
0
0
0
0
0


In [8]:
#df_geographic_unique['reportedZipCode'] = df_geographic_unique['reportedZipCode'].dropna().astype(int).astype(str)
#df_geographic_unique['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_geographic_unique['reportedZipCode']]

df_geographic_unique['censusBlockGroupFips'] = [str(int(float(i))) for i in df_geographic_unique['censusBlockGroupFips']]
df_geographic_unique['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_geographic_unique['censusBlockGroupFips']]

df_geographic_unique['countyCode'] = [str(int(float(i))) for i in df_geographic_unique['countyCode']]
df_geographic_unique['countyCode'] = [censusBG.zfill(5) for censusBG in df_geographic_unique['countyCode']]

df_geographic_unique['censusTract'] = [str(int(float(i))) for i in df_geographic_unique['censusTract']]
df_geographic_unique['censusTract'] = [censusBG.zfill(11) for censusBG in df_geographic_unique['censusTract']]

In [9]:
# Define bins and labels for yearOfLoss_2000_2021
bins_2000_2021 = [0, 2000, 2010, 2020, 2030]
labels_2000_2021 = [1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_2000_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_2000_2021, labels=labels_2000_2021, right=False).astype(int)

# Define bins and labels for yearOfLoss_1990_2021
bins_1990_2021 = [df_geographic_unique['yearOfLoss'].min(), 2000, 2010, 2020, 2030]
labels_1990_2021 = [1990, 2000, 2010, 2020]


#df_geographic_unique['yearOfLoss_1990_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1990_2021, labels=labels_1990_2021, right=False).astype(int)

#df_geographic_unique = df_geographic_unique.drop(columns='yearOfLoss')

In [10]:
df_geographic_unique.head()

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss,yearOfLoss_2000_2021
0,CA,92056.0,06073,06073018512,060730185123,33.2,-117.3,1998,1990
2,FL,32566.0,12113,12113010815,121130108152,30.4,-86.9,1998,1990
3,SC,29902.0,45013,45013000700,450130007002,32.4,-80.7,1994,1990
4,FL,32940.0,12009,12009063107,120090631073,28.3,-80.7,1996,1990
5,VA,23451.0,51810,51810043600,518100436002,36.9,-76.0,1998,1990


## Read shapefiles

In [11]:
states = pygris.states()

state_df = states[['STUSPS', 'NAME', 'geometry']]

Using the default year of 2021


In [12]:
#Checking if all states found in our dataset are in the US Census Bureau TIGER/Line and cartographic boundary shapefiles

unique_states = df_geographic_unique['state'].unique()
state_STUSPS_unique = state_df['STUSPS'].unique()

np.all(np.isin(unique_states, state_STUSPS_unique))

True

In [13]:
# Read the parquet file
df_read = pd.read_parquet("C:/Users/jorda/Box/Flood Damage PredictionProject/Dataset/lat_long_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
lat_long_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [14]:
chunk_size = 25000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 400000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/jorda/Box/Flood Damage PredictionProject/Dataset/BG_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
BG_df = pd.concat(gdf_list, ignore_index=True)

BG_df['year'] = BG_df['year'].replace({2012: 2010, 2021: 2020})
BG_df = BG_df.drop_duplicates(subset=['GEOID', 'year'])

In [15]:
chunk_size = 25000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 100000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/jorda/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)
    
# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
zipcode_df = pd.concat(gdf_list, ignore_index=True)

zipcode_df['year'] = zipcode_df['year'].replace({2012: 2010, 2021: 2020})
zipcode_df = zipcode_df.drop_duplicates(subset=['ZIPcode', 'year'])

In [16]:
# Read the parquet file
df_read = pd.read_parquet("C:/Users/jorda/Box/Flood Damage PredictionProject/Dataset/County_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
County_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

County_df['year'] = County_df['year'].replace({2012: 2010, 2021: 2020})
County_df = County_df.drop_duplicates(subset=['CountyID', 'year'])

In [17]:
# Read the parquet file
df_read = pd.read_parquet("C:/Users/jorda/Box/Flood Damage PredictionProject/Dataset/Tract_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
Tract_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))
Tract_df['year'] = Tract_df['year'].replace({2012: 2010, 2021: 2020})
Tract_df = Tract_df.drop_duplicates(subset=['censusTractID', 'year'])

## Geometry Intersection creation

In [18]:
state_df.rename(columns={'geometry': 'geometry_state'}, inplace=True)
lat_long_df.rename(columns={'geometry': 'geometry_lat_long'}, inplace=True)
BG_df.rename(columns={'geometry': 'geometry_BG'}, inplace=True)
zipcode_df.rename(columns={'geometry': 'geometry_zipcode'}, inplace=True)
County_df.rename(columns={'geometry': 'geometry_county'}, inplace=True)
Tract_df.rename(columns={'geometry': 'geometry_tract'}, inplace=True)

C:\Users\jorda\AppData\Local\Temp\ipykernel_14596\2417984689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df.rename(columns={'geometry': 'geometry_state'}, inplace=True)


In [19]:
df_geographic_unique

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss,yearOfLoss_2000_2021
0,CA,92056.0,06073,06073018512,060730185123,33.2,-117.3,1998,1990
2,FL,32566.0,12113,12113010815,121130108152,30.4,-86.9,1998,1990
3,SC,29902.0,45013,45013000700,450130007002,32.4,-80.7,1994,1990
4,FL,32940.0,12009,12009063107,120090631073,28.3,-80.7,1996,1990
5,VA,23451.0,51810,51810043600,518100436002,36.9,-76.0,1998,1990
...,...,...,...,...,...,...,...,...,...
2357876,FL,NaN,12037,12037970100,120379701002,29.9,-84.4,2005,2000
2357877,FL,NaN,12037,12037970100,120379701002,29.9,-84.4,2016,2010
2358434,VI,NaN,78030,78030961000,780309610004,18.3,-64.9,2004,2000
2362112,VA,NaN,51570,51570830200,515708302002,37.3,-77.4,2006,2000


In [20]:
# Filter for post 2000

df_geographic_unique = df_geographic_unique[df_geographic_unique['yearOfLoss_2000_2021']!=0]

In [21]:
# Setting the multi-index on lat_long_df
lat_long_df.set_index(['latitude', 'longitude'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_lat_long'] = df_geographic_unique.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])

# Resetting the index of lat_long_df (optional, but good practice)
lat_long_df.reset_index(inplace=True)

In [22]:
BG_df.reset_index(inplace=True)

In [23]:
# Initial mapping with multi-index
BG_df.set_index(['GEOID', 'year'], inplace=True)
df_geographic_unique['geometry_BG'] = df_geographic_unique.set_index(['censusBlockGroupFips', 'yearOfLoss_2000_2021']).index.map(BG_df['geometry_BG'])

# Find rows with NaN values 
na_rows = df_geographic_unique['geometry_BG'].isna()

# Reset index of BG_df
#df_single_index = BG_df.reset_index()

# Sort by 'year' in descending order
#df_single_index = df_single_index.sort_values(by='year', ascending=False)

# Drop duplicates based on the 'GEOID' column. Keep the first occurrence.
#df_single_index = df_single_index.drop_duplicates(subset='GEOID', keep='first')

# Map these rows using only the 'GEOID'
#df_geographic_unique.loc[na_rows, 'geometry_BG'] = df_geographic_unique.loc[na_rows, 'censusBlockGroupFips'].map(df_single_index.set_index('GEOID')['geometry_BG'])

# Resetting the index of BG_df (return to multi-index)
#BG_df.reset_index(inplace=True)

In [24]:
# Initial mapping with multi-index
zipcode_df.set_index(['ZIPcode', 'year'], inplace=True)
df_geographic_unique['geometry_zipcode'] = df_geographic_unique.set_index(['reportedZipCode', 'yearOfLoss_2000_2021']).index.map(zipcode_df['geometry_zipcode'])

# Find rows with NaN values 
na_rows = df_geographic_unique['geometry_zipcode'].isna()

# Reset index of BG_df
df_single_index = zipcode_df.reset_index()

# Sort by 'year' in descending order
df_single_index = df_single_index.sort_values(by='year', ascending=False)

# Drop duplicates based on the 'GEOID' column. Keep the first occurrence.
df_single_index = df_single_index.drop_duplicates(subset='ZIPcode', keep='first')

# Map these rows using only the 'GEOID'
df_geographic_unique.loc[na_rows, 'geometry_zipcode'] = df_geographic_unique.loc[na_rows, 'reportedZipCode'].map(df_single_index.set_index('ZIPcode')['geometry_zipcode'])

# Resetting the index of BG_df (return to multi-index)
zipcode_df.reset_index(inplace=True)

In [25]:
# Setting the multi-index on lat_long_df
state_df.set_index(['STUSPS'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_state'] = df_geographic_unique.set_index(['state']).index.map(state_df['geometry_state'])

# Resetting the index of lat_long_df (optional, but good practice)
state_df.reset_index(inplace=True)

In [26]:
# Initial mapping with multi-index
County_df.set_index(['CountyID', 'year'], inplace=True)
df_geographic_unique['geometry_county'] = df_geographic_unique.set_index(['countyCode', 'yearOfLoss_2000_2021']).index.map(County_df['geometry_county'])

# Find rows with NaN values
na_rows = df_geographic_unique['geometry_county'].isna()

# Reset index of County_df
df_single_index = County_df.reset_index()

# Sort by 'year' in descending order
df_single_index = df_single_index.sort_values(by='year', ascending=False)

# Drop duplicates based on the 'countyCode' column. Keep the first occurrence.
df_single_index = df_single_index.drop_duplicates(subset='CountyID', keep='first')

# Map these rows using only the 'countyCode'
df_geographic_unique.loc[na_rows, 'geometry_county'] = df_geographic_unique.loc[na_rows, 'countyCode'].map(df_single_index.set_index('CountyID')['geometry_county'])

# Resetting the index of County_df (return to multi-index)
County_df.reset_index(inplace=True)

In [27]:
# Initial mapping with multi-index
Tract_df.set_index(['censusTractID', 'year'], inplace=True)
df_geographic_unique['geometry_tract'] = df_geographic_unique.set_index(['censusTract', 'yearOfLoss_2000_2021']).index.map(Tract_df['geometry_tract'])

# Find rows with NaN values
na_rows = df_geographic_unique['geometry_tract'].isna()

# Reset index of County_df
df_single_index = Tract_df.reset_index()

# Sort by 'year' in descending order
df_single_index = df_single_index.sort_values(by='year', ascending=False)

# Drop duplicates based on the 'countyCode' column. Keep the first occurrence.
df_single_index = df_single_index.drop_duplicates(subset='censusTractID', keep='first')

# Map these rows using only the 'countyCode'
df_geographic_unique.loc[na_rows, 'geometry_tract'] = df_geographic_unique.loc[na_rows, 'censusTract'].map(df_single_index.set_index('censusTractID')['geometry_tract'])

# Resetting the index of County_df (return to multi-index)
Tract_df.reset_index(inplace=True)

## Drop rows with missing shapefiles

In [28]:
df_geographic_unique = df_geographic_unique[(df_geographic_unique['geometry_BG'].notna())
                              & (df_geographic_unique['geometry_county'].notna())
                              #& (df_geographic_unique['geometry_zipcode'].notna())
                              & (df_geographic_unique['geometry_tract'].notna())]

## Creating the intersections

In [29]:
import geopandas as gpd
from shapely.geometry import Polygon
from itertools import product
import warnings
warnings.filterwarnings("ignore")

In [30]:
df_geographic_unique.columns

Index(['state', 'reportedZipCode', 'countyCode', 'censusTract',
       'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss',
       'yearOfLoss_2000_2021', 'geometry_lat_long', 'geometry_BG',
       'geometry_zipcode', 'geometry_state', 'geometry_county',
       'geometry_tract'],
      dtype='object')

In [31]:
len(df_geographic_unique)

133097

In [34]:
# Create an empty GeoDataFrame to store the intersection results
new_unit_df = gpd.GeoDataFrame(columns=['reportedZipCode', 'countyCode', 'censusTract',
                                       'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss_2000_2021', 'state', 'geometry_zipcode',
                                       'geometry_county', 'geometry_tract','geometry_BG','geometry_lat_long','state_geometry',
                                         'cbgInconsistent', 'tractInconsistent', 'countyInconsistent', 'stateInconsistent', 'latlongInconsistent', 'multiple', 'noOverlap', 'oneWrong'])

# Iterate through each row in BG_df_1990 and each row in lat_long_df to find intersections
for idx_unit, row_unit in df_geographic_unique.iterrows():
    year = row_unit['yearOfLoss_2000_2021']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    
    # Compute intersection geometry
    intersection_geometry = bg_geometry.intersection(lat_long_geometry).intersection(county_geometry).intersection(state_geometry).intersection(tract_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df = pd.concat([new_unit_df, pd.DataFrame({
            'reportedZipCode': [zipcode],
            'countyCode': [county_id],
            'censusTract': [tract_id],
            'censusBlockGroupFips': [bg_id],
            'latitude': [lat],
            'longitude': [long],
            'year': [year],
            'state': [state],
            'geometry_zipcode': [zipcode_geometry],
            'geometry_county': [county_geometry],
            'geometry_tract': [tract_geometry],
            'geometry_BG': [bg_geometry],
            'geometry_lat_long': [lat_long_geometry],
            'state_geometry': [state_geometry]
        })], ignore_index=True)

In [35]:
len(new_unit_df)

9232

In [37]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['state_geometry']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    

    intersection_geometry = lat_long_geometry.intersection(zipcode_geometry).intersection(county_geometry).intersection(state_geometry).intersection(tract_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit, 'cbgInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit, 'cbgInconsistent'] = 1

        

AttributeError: 'NoneType' object has no attribute 'intersection'

In [ ]:
new_unit_df['cbgInconsistent'].sum()

In [ ]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['state_geometry']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    

    intersection_geometry = bg_geometry.intersection(zipcode_geometry).intersection(county_geometry).intersection(state_geometry).intersection(lat_long_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'tractInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'tractInconsistent'] = 1

In [ ]:
new_unit_df['tractInconsistent'].sum()

In [ ]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['state_geometry']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    

    intersection_geometry = bg_geometry.intersection(zipcode_geometry).intersection(tract_geometry).intersection(state_geometry).intersection(lat_long_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'countyInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'countyInconsistent'] = 1

In [ ]:
new_unit_df['countyInconsistent'].sum()

In [ ]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['state_geometry']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    

    intersection_geometry = bg_geometry.intersection(zipcode_geometry).intersection(tract_geometry).intersection(county_geometry).intersection(lat_long_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'stateInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'stateInconsistent'] = 1

In [ ]:
new_unit_df['stateInconsistent'].sum()

In [ ]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['state_geometry']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    

    intersection_geometry = bg_geometry.intersection(zipcode_geometry).intersection(tract_geometry).intersection(county_geometry).intersection(state_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'latlongInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'latlongInconsistent'] = 1

In [ ]:
new_unit_df['latlongInconsistent'].sum()

In [ ]:
# for idx_unit, row_unit in new_unit_df.iterrows():
#     year = row_unit['year']
#     bg_id = row_unit['censusBlockGroupFips']
#     bg_geometry = row_unit['geometry_BG']
#     tract_id = row_unit['censusTract']
#     tract_geometry = row_unit['geometry_tract']
#     county_id = row_unit['countyCode']
#     county_geometry = row_unit['geometry_county']
#     state = row_unit['state']
#     state_geometry = row_unit['state_geometry']
#     lat_long_geometry = row_unit['geometry_lat_long']
#     lat = row_unit['latitude']
#     long = row_unit['longitude']
#     zipcode_geometry = row_unit['geometry_zipcode']
#     zipcode = row_unit['reportedZipCode']
    

#     intersection_geometry = bg_geometry.intersection(lat_long_geometry).intersection(tract_geometry).intersection(county_geometry).intersection(state_geometry)

#     if intersection_geometry is None or intersection_geometry.is_empty:
#         new_unit_df.at[idx_unit,'zipInconsistent'] = 0
#     else:
#         new_unit_df.at[idx_unit,'zipInconsistent'] = 1

In [ ]:
#print(new_unit_df['zipInconsistent'].sum())

In [ ]:
mask = (new_unit_df['latlongInconsistent'] +   new_unit_df['stateInconsistent'] + new_unit_df['countyInconsistent'] + new_unit_df['tractInconsistent'] + new_unit_df['cbgInconsistent']) > 1
new_unit_df.loc[mask, 'multiple'] = 1
new_unit_df.loc[~mask, 'multiple'] = 0

print(sum(new_unit_df['multiple']))

In [ ]:
new_unit_df.head()

In [ ]:
condition = (new_unit_df['latlongInconsistent'] + 
             new_unit_df['stateInconsistent'] + 
             new_unit_df['countyInconsistent'] + 
             new_unit_df['tractInconsistent'] + 
             new_unit_df['cbgInconsistent'] ) == 0

new_unit_df['noOverlap'] = 0  # Default to 0
new_unit_df.loc[condition, 'noOverlap'] = 1

print(sum(new_unit_df['noOverlap']))

In [ ]:
condition = (new_unit_df['latlongInconsistent'] + 
             new_unit_df['stateInconsistent'] + 
             new_unit_df['countyInconsistent'] + 
             new_unit_df['tractInconsistent'] + 
             new_unit_df['cbgInconsistent'])  == 1

new_unit_df['oneWrong'] = 0  # Default to 0
new_unit_df.loc[condition, 'oneWrong'] = 1

print(sum(new_unit_df['oneWrong']))


In [ ]:
# Initialize the count dictionary for 'Wrong' values
wrong_counts = {
    'lat_long_geometry': 0,
    'state_geometry': 0,
    'county_geometry': 0,
    'tract_geometry': 0,
    'bg_geometry': 0,
    'zip_geometry': 0  # added for zip codes
}

# Iterate through the DataFrame
for idx_unit, row_unit in new_unit_df.iterrows():
    if row_unit['oneWrong'] == 1:
        if row_unit['latlongInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'lat_long_geometry'] = 'Wrong'
            wrong_counts['lat_long_geometry'] += 1
        if row_unit['stateInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'state_geometry'] = 'Wrong'
            wrong_counts['state_geometry'] += 1
        if row_unit['countyInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'county_geometry'] = 'Wrong'
            wrong_counts['county_geometry'] += 1
        if row_unit['tractInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'tract_geometry'] = 'Wrong'
            wrong_counts['tract_geometry'] += 1
        if row_unit['cbgInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'bg_geometry'] = 'Wrong'
            wrong_counts['bg_geometry'] += 1
        if row_unit['zipInconsistent'] == 1:  # added for zip codes
            new_unit_df.at[idx_unit, 'zip_geometry'] = 'Wrong'
            wrong_counts['zip_geometry'] += 1

# Print the 'Wrong' counts
for category, count in wrong_counts.items():
    print(f"{category}: {count} 'Wrong' values")


In [ ]:
new_unit_df.to_csv('adjustedDatasetOne.csv')